# DSE 3260
## Week - 9
### Reg. No - 200968216
### Pratinav Seth

In [ ]:
import gym
import numpy as np


In [ ]:
env = gym.make('CliffWalking-v0')

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.9/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.9/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [ ]:
num_episodes = 100
gamma = 1.0
epsilon = 0.1
num_actions = env.action_space.n
num_states = env.observation_space.n

In [ ]:
Q_mc_es = np.zeros((num_states, num_actions))
returns_sum = np.zeros((num_states, num_actions))
returns_count = np.zeros((num_states, num_actions))

Q_on_policy = np.zeros((num_states, num_actions))
returns_sum_on_policy = np.zeros((num_states, num_actions))
returns_count_on_policy = np.zeros((num_states, num_actions))


In [ ]:
for i_episode in range(num_episodes):
    episode = []
    state = env.reset()
    done = False
    
    while not done:
        action = np.random.choice(num_actions)
        next_state, reward, done, info = env.step(action)
        episode.append((state, action, reward))
        state = next_state
    
    states, actions, rewards = zip(*episode)
    discounts = np.array([gamma**i for i in range(len(rewards)+1)])
    
    for i, state in enumerate(states):
        action = actions[i]
        returns_sum[state][action] += sum(rewards[i:] * discounts[:-(1+i)])
        returns_count[state][action] += 1.0
        Q_mc_es[state][action] = returns_sum[state][action] / returns_count[state][action]

In [ ]:
def epsilon_soft_policy(state, Q, epsilon):
    policy = np.ones(num_actions, dtype=float) * epsilon / num_actions
    best_action = np.argmax(Q[state])
    policy[best_action] += (1.0 - epsilon)
    return policy

In [ ]:
for i_episode in range(num_episodes):
    episode_states = []
    episode_actions = []
    episode_rewards = []
    state = env.reset()
    done = False
    
    while not done:
        policy = epsilon_soft_policy(state, Q_on_policy, epsilon)
        action = np.random.choice(num_actions, p=policy)
        next_state, reward, done, info = env.step(action)
        episode_states.append(state)
        episode_actions.append(action)
        episode_rewards.append(reward)
        state = next_state
    
    discounts = gamma ** np.arange(len(episode_rewards) + 1)
    returns = np.cumsum(episode_rewards[::-1])[::-1] * discounts[:-1]
    for t in range(len(episode_states)):
        state = episode_states[t]
        action = episode_actions[t]
        returns_sum_on_policy[state][action] += returns[t]
        returns_count_on_policy[state][action] += 1
        Q_on_policy[state][action] = returns_sum_on_policy[state][action] / returns_count_on_policy[state][action]

In [ ]:
print("Optimal Policy using Monte Carlo ES:\n", np.argmax(Q_mc_es, axis=1))
print("\nOptimal Policy using on-policy first-visit MC control:\n", np.argmax(Q_on_policy, axis=1))

# Compare  and comment  on  the  performance  of  both  methods  in  terms  of  number  of  steps needed to learn optimal policy and the number of episodes

In general, the Monte Carlo ES approach requires a large number of episodes to converge to the best policy. This is due to the fact that it must visit each state-action pair frequently enough to accurately estimate the Q-values. As a result, learning the optimal policy may necessitate a significant number of events.
<br>
<br>
<br>
First-time policy visit The MC control for epsilon-soft policies generates episodes based on an epsilon-soft policy, which means it chooses the best action with probability = (1-epsilon) and a random action with probability epsilon. This allows the algorithm to explore the state-action space while also utilising the best policy at the time. When compared to the Monte Carlo ES method, the on-policy first-visit MC control method requires fewer episodes to converge. This is due to the fact that it improves the policy gradually while creating episodes and updates the Q-values after each episode.
<br>
<br>
<br>
In terms of the number of steps required to learn the optimal policy, assuming convergence, both techniques should learn the best policy in a similar number of steps. The amount of steps necessary for convergence, however, can vary depending on the problem and the hyperparameters used.
